# Chicago Car Crash Analysis

## Importing Packages

In [1]:
# Importing packages needed for this notebook

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [2]:
# Reading the data in

df_crashes = pd.read_csv("data/Traffic_Crashes_-_Crashes_20231109.csv")
df_vehicles = pd.read_csv("data/Traffic_Crashes_-_Vehicles_20231109.csv")
df_people = pd.read_csv("data/Traffic_Crashes_-_People_20231109.csv")

/var/folders/kl/l56g5k7x0xqgjd17r78jnyxc0000gn/T/ipykernel_28610/98205503.py:4: DtypeWarning: Columns (2,19,21,40,41,42,44,48,49,50,53,55,58,59,61,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vehicles = pd.read_csv("data/Traffic_Crashes_-_Vehicles_20231109.csv")
/var/folders/kl/l56g5k7x0xqgjd17r78jnyxc0000gn/T/ipykernel_28610/98205503.py:5: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df_people = pd.read_csv("data/Traffic_Crashes_-_People_20231109.csv")


In [3]:
# Dropping columns

columns_to_drop_crashes = ['RD_NO', 'CRASH_DATE_EST_I', 'LANE_CNT', 'REPORT_TYPE', 'INTERSECTION_RELATED_I', 'NOT_RIGHT_OF_WAY_I', 'HIT_AND_RUN_I', 'DATE_POLICE_NOTIFIED',
                           'SEC_CONTRIBUTORY_CAUSE', 'STREET_NO', 'STREET_DIRECTION', 'STREET_NAME', 'BEAT_OF_OCCURRENCE', 'PHOTOS_TAKEN_I', 'STATEMENTS_TAKEN_I', 'DOORING_I',
                           'WORK_ZONE_I', 'WORK_ZONE_TYPE', 'WORKERS_PRESENT_I', 'INJURIES_FATAL', 'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING',
                           'INJURIES_REPORTED_NOT_EVIDENT','INJURIES_NO_INDICATION', 'INJURIES_UNKNOWN', 'LATITUDE', 'LONGITUDE', 'LOCATION'
                           ]

columns_to_drop_vehicles = ['RD_NO', 'CRASH_DATE', 'CRASH_UNIT_ID', 'UNIT_NO', 'UNIT_TYPE', 'NUM_PASSENGERS', 'VEHICLE_ID', 'CMRC_VEH_I', 'MAKE', 'MODEL', 'VEHICLE_YEAR',
                            'VEHICLE_DEFECT','VEHICLE_USE', 'TRAVEL_DIRECTION', 'TOWED_I', 'FIRE_I', 'OCCUPANT_CNT', 'EXCEED_SPEED_LIMIT_I', 'TOWED_BY', 'TOWED_TO',
                            'AREA_00_I', 'AREA_01_I', 'AREA_02_I', 'AREA_03_I', 'AREA_04_I', 'AREA_05_I', 'AREA_06_I', 'AREA_07_I', 'AREA_08_I', 'AREA_09_I',
                            'AREA_10_I', 'AREA_11_I', 'AREA_12_I', 'AREA_99_I',  'CMV_ID', 'USDOT_NO', 'CCMC_NO', 'ILCC_NO', 'COMMERCIAL_SRC',
                            'GVWR', 'CARRIER_NAME', 'CARRIER_STATE', 'CARRIER_CITY', 'HAZMAT_PLACARDS_I', 'HAZMAT_NAME','UN_NO', 'HAZMAT_PRESENT_I',
                            'HAZMAT_REPORT_I', 'HAZMAT_REPORT_NO', 'MCS_REPORT_I', 'MCS_REPORT_NO', 'HAZMAT_VIO_CAUSE_CRASH_I', 'MCS_VIO_CAUSE_CRASH_I', 'IDOT_PERMIT_NO',
                            'WIDE_LOAD_I', 'TRAILER1_WIDTH', 'TRAILER2_WIDTH', 'TRAILER1_LENGTH', 'TRAILER2_LENGTH', 'TOTAL_VEHICLE_LENGTH', 'AXLE_CNT', 'VEHICLE_CONFIG', 'CARGO_BODY_TYPE',
                            'LOAD_TYPE', 'HAZMAT_OUT_OF_SERVICE_I', 'MCS_OUT_OF_SERVICE_I', 'HAZMAT_CLASS'
                            ]

columns_to_drop_people = ['RD_NO', 'CRASH_DATE', 'VEHICLE_ID', 'PERSON_ID', 'SEAT_NO', 'CITY', 'STATE', 'ZIPCODE', 'DRIVERS_LICENSE_STATE', 'DRIVERS_LICENSE_CLASS', 'EJECTION',
                          'HOSPITAL', 'EMS_AGENCY', 'EMS_RUN_NO', 'BAC_RESULT VALUE', 'PEDPEDAL_ACTION', 'PEDPEDAL_VISIBILITY', 'PEDPEDAL_LOCATION', 'CELL_PHONE_USE', 'INJURY_CLASSIFICATION'
                          ]

df_crashes = df_crashes.drop(columns_to_drop_crashes, axis=1)
df_vehicles = df_vehicles.drop(columns_to_drop_vehicles, axis=1)
df_people = df_people.drop(columns_to_drop_people, axis=1)

## Creating a Master DataFrame

In [4]:
master_df = pd.merge(df_crashes, pd.merge(df_vehicles, df_people, on="CRASH_RECORD_ID"), on="CRASH_RECORD_ID")

## Info and column names of each table

In [5]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3594423 entries, 0 to 3594422
Data columns (total 35 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   CRASH_RECORD_ID          object 
 1   CRASH_DATE               object 
 2   POSTED_SPEED_LIMIT       int64  
 3   TRAFFIC_CONTROL_DEVICE   object 
 4   DEVICE_CONDITION         object 
 5   WEATHER_CONDITION        object 
 6   LIGHTING_CONDITION       object 
 7   FIRST_CRASH_TYPE         object 
 8   TRAFFICWAY_TYPE          object 
 9   ALIGNMENT                object 
 10  ROADWAY_SURFACE_COND     object 
 11  ROAD_DEFECT              object 
 12  CRASH_TYPE               object 
 13  DAMAGE                   object 
 14  PRIM_CONTRIBUTORY_CAUSE  object 
 15  NUM_UNITS                int64  
 16  MOST_SEVERE_INJURY       object 
 17  INJURIES_TOTAL           float64
 18  CRASH_HOUR               int64  
 19  CRASH_DAY_OF_WEEK        int64  
 20  CRASH_MONTH              int64  
 21  Boundari

## Takeaway

- 3,594,423 observations
- 36 combined columns
- 987 MB combined memory file

## Data Cleaning

In [6]:
# Filtering from 2020 and on

master_df['CRASH_DATE'] = pd.to_datetime(master_df['CRASH_DATE'])
master_df['YEAR'] = master_df['CRASH_DATE'].dt.year
master_df = master_df.loc[master_df['INJURIES_TOTAL'] > 0]
master_df = master_df[
    (master_df["YEAR"] >= 2020) &
    ((master_df["VEHICLE_TYPE"] == "PASSENGER") |
     (master_df["VEHICLE_TYPE"] == "SPORT UTILITY VEHICLE (SUV)") |
     (master_df["VEHICLE_TYPE"] == "VAN/MINI-VAN"))
]
master_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 305598 entries, 14 to 1858292
Data columns (total 36 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   CRASH_RECORD_ID          305598 non-null  object        
 1   CRASH_DATE               305598 non-null  datetime64[ns]
 2   POSTED_SPEED_LIMIT       305598 non-null  int64         
 3   TRAFFIC_CONTROL_DEVICE   305598 non-null  object        
 4   DEVICE_CONDITION         305598 non-null  object        
 5   WEATHER_CONDITION        305598 non-null  object        
 6   LIGHTING_CONDITION       305598 non-null  object        
 7   FIRST_CRASH_TYPE         305598 non-null  object        
 8   TRAFFICWAY_TYPE          305598 non-null  object        
 9   ALIGNMENT                305598 non-null  object        
 10  ROADWAY_SURFACE_COND     305598 non-null  object        
 11  ROAD_DEFECT              305598 non-null  object        
 12  CRASH_TYPE         

## Takeaway

- We filtered the data from year 2020 and after, 

In [7]:
master_df.isna().sum()

CRASH_RECORD_ID                0
CRASH_DATE                     0
POSTED_SPEED_LIMIT             0
TRAFFIC_CONTROL_DEVICE         0
DEVICE_CONDITION               0
WEATHER_CONDITION              0
LIGHTING_CONDITION             0
FIRST_CRASH_TYPE               0
TRAFFICWAY_TYPE                0
ALIGNMENT                      0
ROADWAY_SURFACE_COND           0
ROAD_DEFECT                    0
CRASH_TYPE                     0
DAMAGE                         0
PRIM_CONTRIBUTORY_CAUSE        0
NUM_UNITS                      0
MOST_SEVERE_INJURY             0
INJURIES_TOTAL                 0
CRASH_HOUR                     0
CRASH_DAY_OF_WEEK              0
CRASH_MONTH                    0
Boundaries - ZIP Codes      2128
LIC_PLATE_STATE            12230
VEHICLE_TYPE                   0
MANEUVER                       0
FIRST_CONTACT_POINT           86
PERSON_TYPE                    0
SEX                         7393
AGE                        53949
SAFETY_EQUIPMENT            1633
AIRBAG_DEP

# Andy's Space

In [8]:
defect_dict = {"NO DEFECTS" : "NO DEFECTS", "UNKNOWN": "NO DEFECTS", "WORN SURFACE": "DEFECTS", "OTHER":"DEFECTS","RUT, HOLES": "DEFECTS", "SHOULDER DEFECT":"DEFECTS", "DEBRIS ON ROADWAY": "DEFECTS"}

master_df['ROAD_DEFECT'] = master_df['ROAD_DEFECT'].map(defect_dict)
master_df['ROAD_DEFECT'].value_counts()

ROAD_DEFECT
NO DEFECTS    301095
DEFECTS         4503
Name: count, dtype: int64

In [9]:
master_df["CRASH_TYPE"].value_counts()

CRASH_TYPE
INJURY AND / OR TOW DUE TO CRASH    305506
NO INJURY / DRIVE AWAY                  92
Name: count, dtype: int64

In [10]:
master_df["DAMAGE"].value_counts()
master_df['DAMAGE'] = master_df['DAMAGE'].apply(lambda x: 'OVER $1500' if 'OVER $1,500' in x else "UNDER $1500")
master_df['DAMAGE'].value_counts()

DAMAGE
OVER $1500     259341
UNDER $1500     46257
Name: count, dtype: int64

In [11]:
#talk about this with team
master_df["PRIM_CONTRIBUTORY_CAUSE"].value_counts()

PRIM_CONTRIBUTORY_CAUSE
UNABLE TO DETERMINE                                                                 83621
FAILING TO YIELD RIGHT-OF-WAY                                                       53085
FOLLOWING TOO CLOSELY                                                               26314
FAILING TO REDUCE SPEED TO AVOID CRASH                                              26078
DISREGARDING TRAFFIC SIGNALS                                                        24689
IMPROPER TURNING/NO SIGNAL                                                          11340
IMPROPER OVERTAKING/PASSING                                                          9315
NOT APPLICABLE                                                                       9116
DISREGARDING STOP SIGN                                                               8654
OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER     7296
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                         

In [12]:
prim_cause_dict = { 'FOLLOWING TOO CLOSELY': "HUMAN FACTORS", 'FAILING TO REDUCE SPEED TO AVOID CRASH': "HUMAN FACTORS", 
                   'IMPROPER TURNING/NO SIGNAL': 'HUMAN FACTORS', 'IMPROPER OVERTAKING/PASSING' : 'HUMAN FACTORS',
                   'OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER': 'HUMAN FACTORS',
                   'PHYSICAL CONDITION OF DRIVER': "HUMAN FACTORS", 
                   'UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ARREST IS EFFECTED)': "HUMAN FACTORS",
                   'DISTRACTION - FROM INSIDE VEHICLE':"HUMAN FACTORS", 'DISTRACTION - FROM OUTSIDE VEHICLE': 'HUMAN FACTORS',
                   'TEXTING':"HUMAN FACTORS", 'IMPROPER BACKING':"HUMAN FACTORS", 'CELL PHONE USE OTHER THAN TEXTING': "HUMAN FACTORS",
                   'HAD BEEN DRINKING (USE WHEN ARREST IS NOT MADE)': "HUMAN FACTORS", 
                   'DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATION DEVICE, DVD PLAYER, ETC.)':"HUMAN FACTORS",
                   'VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)': "ENVIRONMENTAL FACTORS", 'ANIMAL': "ENVIRONMENTAL FACTORS",
                   'WEATHER': "ENVIRONMENTAL FACTORS", 'EVASIVE ACTION DUE TO ANIMAL,OBJECT, NONMOTORIST': "ENVIRONMENTAL FACTORS",
                   'FAILING TO YIELD RIGHT-OF-WAY':"SIGNAGE/TRAFFIC CONTROL DEVICES", 'DISREGARDING TRAFFIC SIGNALS': "SIGNAGE/TRAFFIC CONTROL DEVICES",
                   'DISREGARDING STOP SIGN':"SIGNAGE/TRAFFIC CONTROL DEVICES", 'DRIVING ON WRONG SIDE/WRONG WAY':"SIGNAGE/TRAFFIC CONTROL DEVICES",
                   'IMPROPER LANE USAGE': "SIGNAGE/TRAFFIC CONTROL DEVICES", 'DISREGARDING OTHER TRAFFIC SIGNS':"SIGNAGE/TRAFFIC CONTROL DEVICES",
                   'TURNING RIGHT ON RED': "SIGNAGE/TRAFFIC CONTROL DEVICES", 'DISREGARDING ROAD MARKINGS':"SIGNAGE/TRAFFIC CONTROL DEVICES",
                   'DISREGARDING YIELD SIGN': "SIGNAGE/TRAFFIC CONTROL DEVICES", 'RELATED TO BUS STOP': "SIGNAGE/TRAFFIC CONTROL DEVICES",
                   'PASSING STOPPED SCHOOL BUS': "SIGNAGE/TRAFFIC CONTROL DEVICES", 
                   'ROAD CONSTRUCTION/MAINTENANCE': "INFRASTRUCTURE ISSUES", 'ROAD ENGINEERING/SURFACE/MARKING DEFECTS': "INFRASTRUCTURE ISSUES",
                   'OBSTRUCTED CROSSWALKS': "INFRASTRUCTURE ISSUES", 
                   'DRIVING SKILLS/KNOWLEDGE/EXPERIENCE': "MISC FACTORS", 'BICYCLE ADVANCING LEGALLY ON RED LIGHT':"MISC FACTORS",
                   'MOTORCYCLE ADVANCING LEGALLY ON RED LIGHT':"MISC FACTORS", 'EQUIPMENT - VEHICLE CONDITION': "MISC FACTORS",
                   'NOT APPLICABLE': "MISC FACTORS"
              }

master_df['PRIM_CONTRIBUTORY_CAUSE'] = master_df['PRIM_CONTRIBUTORY_CAUSE'].map(prim_cause_dict)
master_df['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

PRIM_CONTRIBUTORY_CAUSE
SIGNAGE/TRAFFIC CONTROL DEVICES    100090
HUMAN FACTORS                       95907
MISC FACTORS                        18567
ENVIRONMENTAL FACTORS                6333
INFRASTRUCTURE ISSUES                 492
Name: count, dtype: int64

<details>
  <summary>**Click Me to Expand Breakdown of PRIM_CONTRIBUTORY_CAUSE**</summary>
    
  **human factors:**
    - 'FOLLOWING TOO CLOSELY', 
    - 'FAILING TO REDUCE SPEED TO AVOID CRASH',
    - 'IMPROPER TURNING/NO SIGNAL',
    - 'IMPROPER OVERTAKING/PASSING', 
    - 'OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER',
    - 'PHYSICAL CONDITION OF DRIVER',
    - 'UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ARREST IS EFFECTED)',
    - 'DISTRACTION - FROM INSIDE VEHICLE',
    - 'DISTRACTION - FROM OUTSIDE VEHICLE',
    - 'TEXTING',
    - 'IMPROPER BACKING',
    - 'CELL PHONE USE OTHER THAN TEXTING',
    - 'HAD BEEN DRINKING (USE WHEN ARREST IS NOT MADE)',
    - 'DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATION DEVICE, DVD PLAYER, ETC.)',

**environmental factors:** 
    - 'VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)',
    - 'ANIMAL', 
    - 'WEATHER',
    - 'EVASIVE ACTION DUE TO ANIMAL,OBJECT, NONMOTORIST',

**traffic control devices/signage:**
    - 'FAILING TO YIELD RIGHT-OF-WAY',
    - 'DISREGARDING TRAFFIC SIGNALS',
    - 'DISREGARDING STOP SIGN',
    - 'DRIVING ON WRONG SIDE/WRONG WAY',
    - 'IMPROPER LANE USAGE',
    - 'DISREGARDING OTHER TRAFFIC SIGNS',
    - 'TURNING RIGHT ON RED',
    - 'DISREGARDING ROAD MARKINGS',
    - 'DISREGARDING YIELD SIGN',
    - 'RELATED TO BUS STOP', 
    - 'PASSING STOPPED SCHOOL BUS',

**infrastructure issues:**
    - 'ROAD CONSTRUCTION/MAINTENANCE', 
    - 'ROAD ENGINEERING/SURFACE/MARKING DEFECTS',
    - 'OBSTRUCTED CROSSWALKS',

**misc factors:**
    - 'DRIVING SKILLS/KNOWLEDGE/EXPERIENCE', 
    - 'BICYCLE ADVANCING LEGALLY ON RED LIGHT', 
    - 'MOTORCYCLE ADVANCING LEGALLY ON RED LIGHT'
    - 'EQUIPMENT - VEHICLE CONDITION',
- 'NOT APPLICABLE'

--
'UNABLE TO DETERMINE', - to be dropped

</details>

In [13]:
master_df["NUM_UNITS"].value_counts()
#in int form already

NUM_UNITS
2     202015
3      61190
4      22690
5       7534
1       5671
6       3596
7       1409
8       1014
9        291
12       131
11        53
14         4
Name: count, dtype: int64

In [14]:
#change to no injury, minor, major, fatality
master_df["MOST_SEVERE_INJURY"].value_counts()

injury_dict = {'NONINCAPACITATING INJURY': 'MINOR INJURIES',
              'REPORTED, NOT EVIDENT': 'MINOR INJURIES', 'INCAPACITATING INJURY' : 'MAJOR INJURIES',
              'FATAL': 'FATAL'}

master_df['SEVERE_INJURY'] = master_df['MOST_SEVERE_INJURY'].map(injury_dict)
master_df.drop(labels="MOST_SEVERE_INJURY", axis=1, inplace=True)
master_df['SEVERE_INJURY'].value_counts()

SEVERE_INJURY
MINOR INJURIES    267484
MAJOR INJURIES     35384
FATAL               2730
Name: count, dtype: int64

In [15]:
master_df["INJURIES_TOTAL"].value_counts()
master_df["CRASH_HOUR"].value_counts()
master_df["CRASH_DAY_OF_WEEK"].value_counts()
master_df["CRASH_MONTH"].value_counts()
#int form already

CRASH_MONTH
7     32695
6     30152
8     29755
10    29626
9     29217
5     28983
4     23074
3     22853
1     22210
2     20199
11    19574
12    17260
Name: count, dtype: int64

In [16]:
master_df["LIC_PLATE_STATE"].value_counts()
master_df.drop(labels="LIC_PLATE_STATE", axis=1, inplace=True)
master_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 305598 entries, 14 to 1858292
Data columns (total 35 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   CRASH_RECORD_ID          305598 non-null  object        
 1   CRASH_DATE               305598 non-null  datetime64[ns]
 2   POSTED_SPEED_LIMIT       305598 non-null  int64         
 3   TRAFFIC_CONTROL_DEVICE   305598 non-null  object        
 4   DEVICE_CONDITION         305598 non-null  object        
 5   WEATHER_CONDITION        305598 non-null  object        
 6   LIGHTING_CONDITION       305598 non-null  object        
 7   FIRST_CRASH_TYPE         305598 non-null  object        
 8   TRAFFICWAY_TYPE          305598 non-null  object        
 9   ALIGNMENT                305598 non-null  object        
 10  ROADWAY_SURFACE_COND     305598 non-null  object        
 11  ROAD_DEFECT              305598 non-null  object        
 12  CRASH_TYPE         

In [17]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 305598 entries, 14 to 1858292
Data columns (total 35 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   CRASH_RECORD_ID          305598 non-null  object        
 1   CRASH_DATE               305598 non-null  datetime64[ns]
 2   POSTED_SPEED_LIMIT       305598 non-null  int64         
 3   TRAFFIC_CONTROL_DEVICE   305598 non-null  object        
 4   DEVICE_CONDITION         305598 non-null  object        
 5   WEATHER_CONDITION        305598 non-null  object        
 6   LIGHTING_CONDITION       305598 non-null  object        
 7   FIRST_CRASH_TYPE         305598 non-null  object        
 8   TRAFFICWAY_TYPE          305598 non-null  object        
 9   ALIGNMENT                305598 non-null  object        
 10  ROADWAY_SURFACE_COND     305598 non-null  object        
 11  ROAD_DEFECT              305598 non-null  object        
 12  CRASH_TYPE         

# Harshitha's Space

In [32]:
# DROP unnecessary columns
# master_df.drop(columns=['CRASH_RECORD_ID', 'CRASH_DATE'], inplace=True)

In [31]:
master_df["YEAR"].value_counts()

YEAR
2021    78859
2022    75797
2023    71624
2020    68767
Name: count, dtype: int64

In [33]:
master_df["POSTED_SPEED_LIMIT"].value_counts()

POSTED_SPEED_LIMIT
30    233677
35     26340
25     13814
20      6460
40      5630
45      3353
15      3342
10      1483
55       333
5        209
0        130
50       108
3         60
39        40
60        31
26        14
7          8
9          4
33         4
24         2
34         2
29         2
2          1
Name: count, dtype: int64

In [34]:
# can bin this into binary, Yes or no
master_df["TRAFFIC_CONTROL_DEVICE"].value_counts()

TRAFFIC_CONTROL_DEVICE
TRAFFIC SIGNAL              126153
NO CONTROLS                 111344
STOP SIGN/FLASHER            46214
UNKNOWN                       6480
OTHER                         1879
YIELD                          693
PEDESTRIAN CROSSING SIGN       475
FLASHING CONTROL SIGNAL        339
DELINEATORS                    308
SCHOOL ZONE                    257
RAILROAD CROSSING GATE         257
OTHER REG. SIGN                236
OTHER WARNING SIGN             168
POLICE/FLAGMAN                 107
OTHER RAILROAD CROSSING         71
RR CROSSING SIGN                40
BICYCLE CROSSING SIGN           18
NO PASSING                       8
Name: count, dtype: int64

In [35]:
master_df['TRAFFIC_CONTROL_DEVICE'] = master_df['TRAFFIC_CONTROL_DEVICE'].apply(lambda x: 'SIGNAL' if 'SIGNAL' in x else x)
master_df['TRAFFIC_CONTROL_DEVICE'].value_counts()

TRAFFIC_CONTROL_DEVICE
SIGNAL                      126492
NO CONTROLS                 111344
STOP SIGN/FLASHER            46214
UNKNOWN                       6480
OTHER                         1879
YIELD                          693
PEDESTRIAN CROSSING SIGN       475
DELINEATORS                    308
SCHOOL ZONE                    257
RAILROAD CROSSING GATE         257
OTHER REG. SIGN                236
OTHER WARNING SIGN             168
POLICE/FLAGMAN                 107
OTHER RAILROAD CROSSING         71
RR CROSSING SIGN                40
BICYCLE CROSSING SIGN           18
NO PASSING                       8
Name: count, dtype: int64

In [36]:
master_df['TRAFFIC_CONTROL_DEVICE'] = master_df['TRAFFIC_CONTROL_DEVICE'].apply(lambda x: 'SIGN' if ' SIGN' in x else x)
master_df['TRAFFIC_CONTROL_DEVICE'].value_counts()

TRAFFIC_CONTROL_DEVICE
SIGNAL                     126492
NO CONTROLS                111344
SIGN                        47151
UNKNOWN                      6480
OTHER                        1879
YIELD                         693
DELINEATORS                   308
RAILROAD CROSSING GATE        257
SCHOOL ZONE                   257
POLICE/FLAGMAN                107
OTHER RAILROAD CROSSING        71
NO PASSING                      8
Name: count, dtype: int64

In [38]:
traffic_dict = {'NO CONTROLS': 'NO CONTROLS', 'SIGNAL':'SIGNAL', 'SIGN': 'SIGN', 'UNKNOWN': 'OTHER',
               'OTHER': 'OTHER', 'LANE USE MARKING': 'SIGN', 'YIELD':'SIGN', 'RAILROAD CROSSING GATE': 'SIGNAL',
               'SCHOOL ZONE': 'SIGN', 'DELINEATORS': 'OTHER', 'POLICE/FLAGMAN': 'OTHER', 
               'OTHER RAILROAD CROSSING': 'SIGNAL', 'NO PASSING': 'SIGN', }
master_df['TRAFFIC_CONTROL_DEVICE'] = master_df['TRAFFIC_CONTROL_DEVICE'].map(traffic_dict)
master_df['TRAFFIC_CONTROL_DEVICE'].value_counts()

TRAFFIC_CONTROL_DEVICE
SIGNAL         126820
NO CONTROLS    111344
SIGN            48109
OTHER            8774
Name: count, dtype: int64

In [40]:
master_df['DEVICE_CONDITION'].value_counts()

DEVICE_CONDITION
FUNCTIONING PROPERLY        158188
NO CONTROLS                 115641
UNKNOWN                      15556
OTHER                         2494
FUNCTIONING IMPROPERLY        1859
NOT FUNCTIONING               1168
WORN REFLECTIVE MATERIAL        98
MISSING                         43
Name: count, dtype: int64

In [42]:
device_dict = {'NO CONTROLS': 'NO CONTROLS',
               'FUNCTIONING PROPERLY': 'FUNCTIONING PROPERLY',
               'UNKNOWN': 'NO CONTROLS',
               'FUNCTIONING IMPROPERLY':'FUNCTIONING IMPROPERLY/ MISSING',
               'NOT FUNCTIONING':'FUNCTIONING IMPROPERLY/ MISSING',
               'WORN REFLECTIVE MATERIAL': 'FUNCTIONING PROPERLY',
               'MISSING': 'FUNCTIONING IMPROPERLY/ MISSING',
               'OTHER': 'FUNCTIONING PROPERLY'}
master_df['DEVICE_CONDITION'] = master_df['DEVICE_CONDITION'].map(device_dict)
master_df['DEVICE_CONDITION'].value_counts()

DEVICE_CONDITION
FUNCTIONING PROPERLY               160780
NO CONTROLS                        131197
FUNCTIONING IMPROPERLY/ MISSING      3070
Name: count, dtype: int64

In [44]:
# bin into clear or not clear? unknown = clear, other=clear 
master_df["WEATHER_CONDITION"].value_counts()

WEATHER_CONDITION
CLEAR                       241669
RAIN                         29409
CLOUDY/OVERCAST               8640
SNOW                          7626
UNKNOWN                       4299
OTHER                         1194
FREEZING RAIN/DRIZZLE         1134
FOG/SMOKE/HAZE                 379
BLOWING SNOW                   377
SLEET/HAIL                     257
SEVERE CROSS WIND GATE          61
BLOWING SAND, SOIL, DIRT         2
Name: count, dtype: int64

In [46]:
weather_dict = {'CLEAR': 'CLEAR', 'RAIN': 'INCLEMENT', 'CLOUDY/OVERCAST': 'INCLEMENT', 'SNOW': 'INCLEMENT', 'UNKNOWN': 'CLEAR', 'OTHER': 'CLEAR',
                'FREEZING RAIN/DRIZZLE': 'INCLEMENT', 'BLOWING SNOW':'INCLEMENT', 'FOG/SMOKE/HAZE': 'INCLEMENT', 'SLEET/HAIL': 'INCLEMENT', 'SEVERE CROSS WIND GATE': 'INCLEMENT',
                'BLOWING SAND, SOIL, DIRT': 'INCLEMENT'
               }
master_df['WEATHER_CONDITION'] = master_df['WEATHER_CONDITION'].map(weather_dict)
master_df['WEATHER_CONDITION'].value_counts()

WEATHER_CONDITION
CLEAR        247162
INCLEMENT     47885
Name: count, dtype: int64

In [48]:
master_df["LIGHTING_CONDITION"].value_counts()

LIGHTING_CONDITION
DAYLIGHT                  182246
DARKNESS, LIGHTED ROAD     85117
DARKNESS                   11130
DUSK                        9077
DAWN                        4836
UNKNOWN                     2641
Name: count, dtype: int64

In [50]:
lighting_dict = {'DAYLIGHT':'VISIBLE','DARKNESS, LIGHTED ROAD':'DIMINISHED VISIBILITY', 'DARKNESS':'DIMINISHED VISIBILITY',
                'DUSK':'DIMINISHED VISIBILITY', 'DAWN': 'DIMINISHED VISIBILITY', 'UNKNOWN': 'VISIBLE'}
master_df['LIGHTING_CONDITION'] = master_df['LIGHTING_CONDITION'].map(lighting_dict)
master_df['LIGHTING_CONDITION'].value_counts()

LIGHTING_CONDITION
VISIBLE                  184887
DIMINISHED VISIBILITY    110160
Name: count, dtype: int64

In [53]:
# Looking at the FIRST_CRASH_TYPE column 
master_df['FIRST_CRASH_TYPE'].value_counts()

FIRST_CRASH_TYPE
REAR END                        76426
ANGLE                           71873
TURNING                         62795
SIDESWIPE SAME DIRECTION        21119
PARKED MOTOR VEHICLE            17153
PEDESTRIAN                      13729
HEAD ON                          8609
PEDALCYCLIST                     8233
FIXED OBJECT                     6004
SIDESWIPE OPPOSITE DIRECTION     4714
REAR TO FRONT                    1436
REAR TO SIDE                     1425
OTHER OBJECT                     1039
OVERTURNED                        217
OTHER NONCOLLISION                133
REAR TO REAR                       87
ANIMAL                             28
TRAIN                              27
Name: count, dtype: int64

In [54]:
first_crash_dict = {'REAR END': 'REAR END', 'SIDESWIPE SAME DIRECTION': 'SIDE SWIPE', 'TURNING': 'TURNING', 'PARKED MOTOR VEHICLE': 'OBJECT', 'ANGLE': 'ANGLE', 'PEDESTRIAN': 'PERSON OR ANIMAL',
                    'SIDESWIPE OPPOSITE DIRECTION': 'SIDE SWIPE', 'FIXED OBJECT': 'OBJECT', 'PEDALCYCLIST': 'PERSON OR ANIMAL', 'HEAD ON': 'OTHER', 'REAR TO FRONT': 'OTHER',
                    'REAR TO SIDE': 'OTHER', 'OTHER OBJECT': 'OBJECT', 'REAR TO REAR': 'OTHER', 'OTHER NONCOLLISION': 'OTHER', 'ANIMAL': 'PERSON OR ANIMAL', 'OVERTURNED': 'OTHER', 'TRAIN': 'OTHER'
                    }
master_df['FIRST_CRASH_TYPE'] = master_df['FIRST_CRASH_TYPE'].map(first_crash_dict)
master_df['FIRST_CRASH_TYPE'].value_counts()

FIRST_CRASH_TYPE
REAR END            76426
ANGLE               71873
TURNING             62795
SIDE SWIPE          25833
OBJECT              24196
PERSON OR ANIMAL    21990
OTHER               11934
Name: count, dtype: int64

In [55]:
master_df['TRAFFICWAY_TYPE'].value_counts()

TRAFFICWAY_TYPE
NOT DIVIDED                        112430
FOUR WAY                            64229
DIVIDED - W/MEDIAN (NOT RAISED)     46579
DIVIDED - W/MEDIAN BARRIER          21970
ONE-WAY                             17223
T-INTERSECTION                      10511
OTHER                                5054
PARKING LOT                          3653
CENTER TURN LANE                     2891
UNKNOWN INTERSECTION TYPE            2364
ALLEY                                1727
FIVE POINT, OR MORE                  1175
Y-INTERSECTION                       1155
TRAFFIC ROUTE                        1026
UNKNOWN                               959
NOT REPORTED                          822
RAMP                                  614
DRIVEWAY                              426
L-INTERSECTION                        152
ROUNDABOUT                             87
Name: count, dtype: int64

In [56]:
#  
trafficway_dict = {'NOT DIVIDED': 'NOT DIVIDED', 'DIVIDED - W/MEDIAN (NOT RAISED)': 'DIVIDED', 'ONE-WAY': 'OTHER', 'FOUR WAY': 'OTHER', 'DIVIDED - W/MEDIAN BARRIER': 'DIVIDED',
                    'PARKING LOT': 'OTHER', 'OTHER': 'OTHER', 'T-INTERSECTION': 'INTERSECTION', 'ALLEY': 'OTHER', 'UNKNOWN': 'NOT DIVIDED', 'CENTER TURN LANE': 'OTHER', 
                    'UNKNOWN INTERSECTION TYPE': 'INTERSECTION', 'RAMP': 'OTHER', 'DRIVEWAY': 'OTHER', 'FIVE POINT, OR MORE': 'OTHER', 'Y-INTERSECTION': 'INTERSECTION', 
                    'TRAFFIC ROUTE': 'OTHER', 'NOT REPORTED': 'OTHER', 'ROUNDABOUT': 'INTERSECTION', 'L-INTERSECTION': 'INTERSECTION'
               }
master_df['TRAFFICWAY_TYPE'] = master_df['TRAFFICWAY_TYPE'].map(trafficway_dict)
master_df['TRAFFICWAY_TYPE'].value_counts()

TRAFFICWAY_TYPE
NOT DIVIDED     113389
OTHER            98840
DIVIDED          68549
INTERSECTION     14269
Name: count, dtype: int64

In [57]:
# Street alignment at crash location, as determined by reporting officer. maybe drop?
alignment_dict = {'STRAIGHT AND LEVEL': 'STRAIGHT', 'STRAIGHT ON GRADE': 'STRAIGHT', 'CURVE, LEVEL': 'NOT STRAIGHT', 'STRAIGHT ON HILLCREST': 'STRAIGHT',
                    'CURVE ON GRADE': 'NOT STRAIGHT', 'CURVE ON HILLCREST': 'NOT STRAIGHT'
               }
master_df['ALIGNMENT'] = master_df['ALIGNMENT'].map(alignment_dict)
master_df['ALIGNMENT'].value_counts()

ALIGNMENT
STRAIGHT        291999
NOT STRAIGHT      3048
Name: count, dtype: int64

In [58]:
roadway_surface_dict = {'DRY': 'DRY', 'WET': 'NOT DRY', 'UNKNOWN': 'DRY', 'SNOW OR SLUSH': 'NOT DRY', 'ICE': 'NOT DRY', 
                'OTHER': 'OTHER', 'SAND, MUD, DIRT': 'NOT DRY'
               }
master_df['ROADWAY_SURFACE_COND'] = master_df['ROADWAY_SURFACE_COND'].map(roadway_surface_dict)
master_df['ROADWAY_SURFACE_COND'].value_counts()

ROADWAY_SURFACE_COND
DRY        240911
NOT DRY     53347
OTHER         789
Name: count, dtype: int64

# Dre's Space

In [18]:
## Vehicle Type

master_df['VEHICLE_TYPE'].value_counts()

VEHICLE_TYPE
PASSENGER                      238025
SPORT UTILITY VEHICLE (SUV)     51648
VAN/MINI-VAN                    15925
Name: count, dtype: int64

In [19]:
# Maneuver

MANEUVER_dict = {'STRAIGHT AHEAD': 'STRAIGHT', 'TURNING LEFT': 'OTHER', 'SLOW/STOP IN TRAFFIC': 'OTHER', 'PARKED': 'OTHER', 'TURNING RIGHT': 'OTHER', 'UNKNOWN/NA': 'STRAIGHT',
                 'OTHER': 'OTHER', 'PASSING/OVERTAKING': 'OTHER', 'CHANGING LANES': 'OTHER', 'U-TURN': 'OTHER', 'BACKING': 'OTHER', 'STARTING IN TRAFFIC': 'OTHER', 
                 'AVOIDING VEHICLES/OBJECTS': 'OTHER', 'ENTERING TRAFFIC LANE FROM PARKING': 'OTHER', 'SKIDDING/CONTROL LOSS': 'OTHER', 'DRIVING WRONG WAY': 'OTHER',
                 'MERGING': 'OTHER', 'ENTER FROM DRIVE/ALLEY': 'OTHER', 'PARKED IN TRAFFIC LANE': 'OTHER', 'NEGOTIATING A CURVE': 'OTHER', 'LEAVING TRAFFIC LANE TO PARK': 'OTHER',
                 'TURNING ON RED': 'OTHER', 'DISABLED': 'OTHER', 'DRIVERLESS': 'OTHER', 'DIVERGING': 'OTHER'}

master_df['MANEUVER'] = master_df['MANEUVER'].map(MANEUVER_dict)

In [20]:

master_df['MANEUVER'].value_counts()

MANEUVER
STRAIGHT    194291
OTHER       111307
Name: count, dtype: int64

In [21]:
# FIRST CONTACT POINT

master_df['FIRST_CONTACT_POINT'] = master_df['FIRST_CONTACT_POINT'].fillna('FRONT')

FIRST_CONTACT_POINT_dict = {'FRONT': 'FRONT', 'REAR': 'REAR', 'FRONT-LEFT-CORNER': 'FRONT', 'FRONT-RIGHT-CORNER': 'FRONT', 'SIDE-LEFT': 'SIDES', 'SIDE-RIGHT': 'SIDES', 
                            'REAR-LEFT-CORNER': 'REAR',
                            'REAR-RIGHT-CORNER':'REAR', 'SIDE-LEFT-REAR': 'SIDES', 'SIDE-LEFT-FRONT':'SIDES', 'SIDE-RIGHT-REAR':'SIDES', 'SIDE-RIGHT-FRONT': 'SIDES',
                            'TOTAL (ALL AREAS)': 'OTHER', 'NONE': 'NONE', 'OTHER': 'OTHER', 'UNDER CARRIAGE': 'OTHER', 'TOP': 'OTHER', 'ROOF': 'OTHER', 'UNKNOWN': 'FRONT',
                            'REAR-LEFT': 'REAR', 'FRONT-RIGHT': 'FRONT'}

master_df['FIRST_CONTACT_POINT'] = master_df['FIRST_CONTACT_POINT'].map(FIRST_CONTACT_POINT_dict)

In [22]:
master_df['FIRST_CONTACT_POINT'].value_counts(dropna=False)

FIRST_CONTACT_POINT
FRONT    179686
REAR      63448
SIDES     57061
OTHER      4030
NONE       1373
Name: count, dtype: int64

In [23]:
# Person Type

PERSON_TYPE_dict = {'DRIVER': 'DRIVER', 'PASSENGER': 'PASSENGER', 'PEDESTRIAN': 'OTHER', 'BICYCLE': 'OTHER', 'NON-MOTOR VEHICLE': 'OTHER', 'NON-CONTACT VEHICLE': 'OTHER'}

master_df['PERSON_TYPE'] = master_df['PERSON_TYPE'].map(PERSON_TYPE_dict)

In [24]:
master_df['PERSON_TYPE'].value_counts(dropna=False)

PERSON_TYPE
DRIVER       198143
PASSENGER     95401
OTHER         12054
Name: count, dtype: int64

In [25]:
# Sex

master_df = master_df[master_df['SEX'] != 'X']

master_df['SEX'].value_counts(dropna=False)

SEX
M      157656
F      129998
NaN      7393
Name: count, dtype: int64

In [26]:
# Age

master_df['AGE'].value_counts()

# Filter for ages 16-75??

AGE
25.0     7372
23.0     7232
27.0     7173
24.0     7132
28.0     7108
         ... 
101.0       4
97.0        2
108.0       2
110.0       2
105.0       1
Name: count, Length: 107, dtype: int64

In [27]:
# Safety Equipment

master_df['SAFETY_EQUIPMENT'] = master_df['SAFETY_EQUIPMENT'].fillna('USAGE UNKNOWN')

SAFETY_EQUIPMENT_dict = {'USAGE UNKNOWN': 'UNKNOWN', 'SAFETY BELT USED': 'USED', 'NONE PRESENT': 'NONE', 'HELMET NOT USED': 'NOT USED', 'SAFETY BELT NOT USED': 'NOT USED',
                         'CHILD RESTRAINT - FORWARD FACING': 'USED', 'BICYCLE HELMET (PEDACYCLIST INVOLVED ONLY)': 'USED', 'CHILD RESTRAINT - TYPE UNKNOWN': 'USED',
                         'CHILD RESTRAINT - REAR FACING': 'USED', 'DOT COMPLIANT MOTORCYCLE HELMET': 'USED', 'BOOSTER SEAT': 'USED', 'CHILD RESTRAINT NOT USED': 'NOT USED', 
                         'CHILD RESTRAINT USED IMPROPERLY': 'USED', 'NOT DOT COMPLIANT MOTORCYCLE HELMET': 'USED', 'WHEELCHAIR': 'USED', 'SHOULD/LAP BELT USED IMPROPERLY': 'USED',
                         'HELMET USED': 'USED', 'STRETCHER': 'USED', 'CHILD RESTRAINT USED': 'USED'}

master_df['SAFETY_EQUIPMENT'] = master_df['SAFETY_EQUIPMENT'].map(SAFETY_EQUIPMENT_dict)

In [28]:
master_df['SAFETY_EQUIPMENT'].value_counts(dropna=False)

SAFETY_EQUIPMENT
UNKNOWN     139877
USED        136400
NONE         12606
NOT USED      6164
Name: count, dtype: int64

In [29]:
# Airbag Deployed

master_df['AIRBAG_DEPLOYED'] = master_df['AIRBAG_DEPLOYED'].fillna('DID NOT DEPLOY')

AIRBAG_DEPLOYED_dict = {'DID NOT DEPLOY': 'DID NOT DEPLOY', 'NOT APPLICABLE': 'NOT APPLICABLE', 'DEPLOYED, COMBINATION': 'DEPLOYED', 'DEPLOYMENT UNKNOWN': 'UNKNOWN', 
                        'DEPLOYED, FRONT': 'DEPLOYED', 'DEPLOYED, SIDE': 'DEPLOYED', 'DEPLOYED OTHER (KNEE, AIR, BELT, ETC.)': 'DEPLOYED'}

master_df['AIRBAG_DEPLOYED'] = master_df['AIRBAG_DEPLOYED'].map(AIRBAG_DEPLOYED_dict)

In [30]:
master_df['AIRBAG_DEPLOYED'].value_counts(dropna=False)

AIRBAG_DEPLOYED
DID NOT DEPLOY    115041
DEPLOYED           83943
NOT APPLICABLE     65199
UNKNOWN            30864
Name: count, dtype: int64

In [37]:
# Driver Action

master_df['DRIVER_ACTION'] = master_df['DRIVER_ACTION'].fillna('NONE')

DRIVER_ACTION_dict = {'NONE': 'NO ACTION TAKEN', 'UNKNOWN': 'NO ACTION TAKEN', 'FAILED TO YIELD': 'ACTIONS TAKEN', 'OTHER': 'ACTIONS TAKEN', 'DISREGARDED CONTROL DEVICES': 'ACTIONS TAKEN',
                      'FOLLOWED TOO CLOSELY': 'ACTIONS TAKEN', 'TOO FAST FOR CONDITIONS': 'ACTIONS TAKEN', 'IMPROPER TURN': 'ACTIONS TAKEN', 'IMPROPER LANE CHANGE': 'ACTIONS TAKEN',
                      'IMPROPER PASSING': 'ACTIONS TAKEN', 'WRONG WAY/SIDE': 'ACTIONS TAKEN', 'IMPROPER BACKING': 'ACTIONS TAKEN', 'OVERCORRECTED': 'ACTIONS TAKEN', 'EVADING POLICE VEHICLE': 'ACTIONS TAKEN',
                      'CELL PHONE USE OTHER THAN TEXTING': 'ACTIONS TAKEN', 'EMERGENCY VEHICLE ON CALL': 'ACTIONS TAKEN', 'IMPROPER PARKING': 'ACTIONS TAKEN', 'TEXTING': 'ACTIONS TAKEN',
                      'LICENSE RESTRICTIONS': 'ACTIONS TAKEN', 'STOPPED SCHOOL BUS': 'ACTIONS TAKEN'}

master_df['DRIVER_ACTION'] = master_df['DRIVER_ACTION'].map(DRIVER_ACTION_dict)

In [39]:
master_df['DRIVER_ACTION'].value_counts(dropna=False)

DRIVER_ACTION
NO ACTION TAKEN    216815
ACTIONS TAKEN       78232
Name: count, dtype: int64

In [41]:
# Driver Vision

master_df['DRIVER_VISION'] = master_df['DRIVER_VISION'].fillna('NONE')

DRIVER_VISION_dict = { 'NONE':'UNKNOWN', 'NOT OBSCURED': 'NOT OBSCURED', 'UNKNOWN': 'UNKNOWN', 'OTHER': 'OBSCURED', 'MOVING VEHICLES': 'OBSCURED', 'PARKED VEHICLES': 'OBSCURED',
                      'WINDSHIELD (WATER/ICE)': 'OBSCURED', 'BLINDED - SUNLIGHT': 'OBSCURED', 'TREES, PLANTS': 'OBSCURED', 'BUILDINGS': 'OBSCURED',
                      'BLINDED - HEADLIGHTS': 'OBSCURED', 'BLOWING MATERIALS': 'OBSCURED', 'HILLCREST': 'OBSCURED',
                      'EMBANKMENT': 'OBSCURED', 'SIGNBOARD': 'OBSCURED'}

master_df['DRIVER_VISION'] = master_df['DRIVER_VISION'].map(DRIVER_VISION_dict)

In [43]:
master_df['DRIVER_VISION'].value_counts(dropna=False)

DRIVER_VISION
UNKNOWN         182906
NOT OBSCURED    105706
OBSCURED          6435
Name: count, dtype: int64

In [45]:
# Physical Condition

master_df['PHYSICAL_CONDITION'] = master_df['PHYSICAL_CONDITION'].fillna('UNKNOWN')

PHYSICAL_CONDITION_dict = {'UNKNOWN': 'UNKNOWN', 'NORMAL': 'NORMAL', 'REMOVED BY EMS': 'NOT NORMAL', 'IMPAIRED - ALCOHOL': 'NOT NORMAL', 'OTHER': 'NOT NORMAL', 'EMOTIONAL': 'NOT NORMAL',
                           'FATIGUED/ASLEEP': 'NOT NORMAL', 'ILLNESS/FAINTED': 'NOT NORMAL', 'IMPAIRED - DRUGS': 'NOT NORMAL', 'HAD BEEN DRINKING': 'NOT NORMAL', 
                           'IMPAIRED - ALCOHOL AND DRUGS': 'NOT NORMAL', 'MEDICATED': 'NOT NORMAL'}

master_df['PHYSICAL_CONDITION'] = master_df['PHYSICAL_CONDITION'].map(PHYSICAL_CONDITION_dict)

In [47]:
master_df['PHYSICAL_CONDITION'].value_counts(dropna=False)

PHYSICAL_CONDITION
NORMAL        146803
UNKNOWN       137339
NOT NORMAL     10905
Name: count, dtype: int64

In [49]:
# BAC Result

master_df['BAC_RESULT'] = master_df['BAC_RESULT'].fillna('UNKNOWN')

BAC_RESULT_dict = {'TEST NOT OFFERED': 'TEST NOT OFFERED', 'TEST REFUSED': 'TEST REFUSED', 'TEST PERFORMED, RESULTS UNKNOWN': 'TEST PERFORMED',
                   'TEST TAKEN': 'TEST PERFORMED'}

master_df['BAC_RESULT'] = master_df['BAC_RESULT'].map(BAC_RESULT_dict)

In [51]:
master_df['BAC_RESULT'].value_counts()

BAC_RESULT
TEST NOT OFFERED    195341
TEST REFUSED          2702
TEST PERFORMED        1955
Name: count, dtype: int64